# Sleep Analysis

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
#%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
sns.set(style='ticks', context='notebook')
#plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['figure.figsize'] = (10,5)
plt.close('all')

palette = bp.colors.fau_palette
sns.set_palette(palette)
palette

In [ ]:
pg.options['round'] = 3

In [ ]:
export_path = Path("../../exports")

## Import

In [ ]:
imu_data = pd.read_csv(export_path.joinpath("imu_features_complete.csv"), index_col=['subject', 'night'])
imu_data.head()

Load Cortisol Data and use the as index for the IMU data to join on

In [ ]:
cort_samples = pd.read_csv(export_path.joinpath("cortisol_samples_cleaned.csv"))
# assign each night an unique_id to allow repeated measures analyses and insert into dataframe
cort_samples.insert(2, 'night_id', cort_samples['subject'] + '_' + cort_samples['night'].astype(str))
cort_samples = cort_samples.drop(columns=['time'])
cort_samples = cort_samples.set_index(list(cort_samples.columns.drop('cortisol')))

In [ ]:
data_index = cort_samples.drop(columns='cortisol').unstack()
data_index.columns = []
imu_data = data_index.join(imu_data, how='inner')
imu_data.columns.name = 'feature'
imu_data = pd.DataFrame(imu_data.stack(), columns=["imu"])

In [ ]:
imu_data.head()

## Plots and Statistics

In [ ]:
order = ['Spontaneous', 'Known Alarm', 'Unknown Alarm']

### Weekend vs. Weekday

Condition per Weekday Type

In [ ]:
fig, ax = plt.subplots()
df_nights = pd.DataFrame(imu_data.groupby(["condition", "weekend"]).size(), columns=["nights"])
df_nights = df_nights.groupby('condition').apply(lambda x: 100 * (x / x.sum())).T.stack().T
df_nights.columns = df_nights.columns.droplevel(0)

df_nights = df_nights.reindex(order)

display(df_nights.T)

ax = df_nights.plot(kind='bar', stacked=True, ax=ax, rot=0)
ax.legend().set_title(None)
ax.set_ylabel("Recorded CARs [%]")
fig.tight_layout()

In [ ]:
hue = 'feature'

dv = 'imu'
between = 'weekend'
group = 'weekend'

In [ ]:
df_stats = imu_data.unstack().filter(like="last").filter(like="60").stack()
df_stats = df_stats.loc[~(df_stats.groupby(hue)[dv].apply(lambda df: (df - df.mean())/df.std()) > 3)]

df_norm = df_stats.reset_index().groupby(hue).apply(lambda df: pg.normality(data=df, dv=dv, group=group))
df_anova = df_stats.reset_index().groupby(hue).apply(lambda df: pg.kruskal(data=df, dv=dv, between=between))
#df_anova['p-corr'] = pg.multicomp(df_anova['p-unc'].values, method='fdr_bh')[1]

#display(df_norm)
display(df_anova)

In [ ]:
x = 'weekend'
y = 'ss_max_60_last_hour'

In [ ]:
fig, ax = plt.subplots()
df_plot = df_stats.unstack()[dv]

sns.boxplot(data=df_plot.reset_index(), x=x, y=y, ax=ax, notch=True)
fig.tight_layout()

### Condition

In [ ]:
order = ['Spontaneous', 'Known Alarm', 'Unknown Alarm']

In [ ]:
hue = 'feature'

dv = 'imu'
between = 'condition'
group = 'condition'

In [ ]:
df_stats = imu_data.unstack().filter(like="last").stack()
df_stats = df_stats.loc[~(df_stats.groupby(hue)[dv].apply(lambda df: (df - df.mean())/df.std()) > 3)]

df_norm = df_stats.reset_index().groupby(hue).apply(lambda df: pg.normality(data=df, dv=dv, group=group))
df_anova = df_stats.reset_index().groupby(hue).apply(lambda df: pg.anova(data=df, dv=dv, between=between))

display(df_norm.T)
display(df_anova)

In [ ]:
x = 'condition'
y = 'ss_number_60_last_hour'

In [ ]:
fig, ax = plt.subplots()
df_plot = df_stats.unstack()[dv]

sns.boxplot(data=df_plot.reset_index(), x=x, y=y, ax=ax, order=order, notch=True)
fig.tight_layout()

### Pairplots

In [ ]:
df_pairplot = imu_data.unstack('feature')['imu'].filter(like="last_hour").reset_index('condition')
df_pairplot = df_pairplot.drop(columns=df_pairplot.filter(like="60").columns)
df_pairplot = df_pairplot.drop(columns=df_pairplot.filter(like="max_position").columns)

g = sns.pairplot(data=df_pairplot, hue='condition', corner=True)
g = g.map_lower(sns.kdeplot, levels=3, color=".2", alpha=0.5)